##                                                Entraînement du Modéle IA
### Importation des modules 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import joblib
### A - Chargement de données d'entraînement
# Charger le dataset
data = pd.read_csv('train.csv')

# Afficher les premières lignes du dataset
data.head()
Nous allons supprimer les colonnes qui sont unitile pour l'entrainement de notre modele. Dans notre dataset nous juste la colonne name qui est unitile. 
data.drop(columns=["Name"], inplace=True)

### B - Préparation des données

#### 1- Gestion des variables manquantes
Nous allons d'abord vérifier les variables manquantes dans les données en utilisant isnull().
data.isnull().sum()
for col in ['HomePlanet', 'Destination', 'VIP']:
    data[col] = data[col].fillna(data[col].mode()[0])

data['Age'] = data['Age'].fillna(data['Age'].median())


cols_depenses = ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
data.loc[data['CryoSleep'].isna() & (data[cols_depenses].sum(axis=1) > 0), 'CryoSleep'] = False




for col in cols_depenses:
    mediane = data[col].median()
    data[col] = data[col].fillna(mediane)
data.isnull().sum()
colonnes_cibles = ["CryoSleep", "Cabin"]
data_cleaned = data.dropna(subset=colonnes_cibles)

data_cleaned.info()
#### 2- Transformation des données
data_cleaned['CryoSleep'] = data_cleaned['CryoSleep'].astype(bool)


data_cleaned.info()

data_cleaned = data_cleaned.copy()

data_cleaned['GroupId'] = data_cleaned['PassengerId'].apply(lambda x: x.split('_')[0])
group_sizes = data_cleaned['GroupId'].value_counts()
data_cleaned['GroupSize'] = data_cleaned['GroupId'].map(group_sizes)
data_cleaned.drop(["GroupId", "PassengerId"], axis=1, inplace=True)

data_cleaned[['Deck','Cabin_Num','Side']]=data_cleaned['Cabin'].str.split("/",expand = True)
data_cleaned.drop(['Cabin'], axis=1, inplace=True)

cols_depenses = ['RoomService','FoodCourt','Spa','VRDeck']
data_cleaned['TotalSpend'] = data_cleaned[cols_depenses].sum(axis =1)

data_cleaned.head()

colonne =  ["CryoSleep","VIP","Transported","Cabin_Num"]
data_cleaned[colonne] = data_cleaned[colonne].astype(int)

data_cleaned.info()
colonne =  ["Cabin_Num","GroupSize"]
data_cleaned[colonne] = data_cleaned[colonne].astype(float)

data_cleaned.info()
## C - Encodage et normalisation
### 1 - Encodage des variables categorielles
categorical_cols = ['HomePlanet', 'Destination', 'Deck', 'Side']
data_encoded = pd.get_dummies(data_cleaned, columns=categorical_cols, drop_first=False)

bool_columns = data_encoded.select_dtypes(include='bool').columns
data_encoded[bool_columns] = data_encoded[bool_columns].astype(int)

data_encoded.head()
### 2 - Normalisation des variables numeriques
numeric_cols = [
    'Age', 'RoomService', 'FoodCourt', 'ShoppingMall',
    'Spa', 'VRDeck', 'Cabin_Num', 'GroupSize', 'TotalSpend'
]

scaler = StandardScaler()
data_encoded[numeric_cols] = scaler.fit_transform(data_encoded[numeric_cols])

data_encoded.head()
Je récupère le scaler utilisé pour la normalisation afin de l'utiliser dans mon interface Streamlit.
colonnes_modele = data_encoded.columns.tolist()
joblib.dump(colonnes_modele, "colonnes_modele.joblib")

joblib.dump(scaler , "scarler2.joblib")
Je sépare les variables caractéristiques de la variable cible.

cible = data_encoded['Transported']
carac = data_encoded.drop(columns=['Transported'], axis=1)  

# D - Modèles
Les données ont été divisées en un jeu d'entraînement et un jeu de test pour évaluer les performances du modèle.
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(carac, cible, test_size=0.3, random_state=42)
### 1- Régerssion Logistique
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score, roc_curve

# Initialisation et entraînement du modèle

log_reg = LogisticRegression(random_state=42)
log_reg.fit(X_train, y_train)

# Prédictions
y_pred_log_reg = log_reg.predict(X_test)

# Matrice de confusion
conf_matrix_log_reg = confusion_matrix(y_test, y_pred_log_reg)
print("Matrice de Confusion (Régression Logistique) :\n", conf_matrix_log_reg)

# Rapport de classification
class_report_log_reg = classification_report(y_test, y_pred_log_reg)
print("Rapport de Classification (Régression Logistique) :\n", class_report_log_reg)

# AUC-ROC
roc_auc_log_reg = roc_auc_score(y_test, y_pred_log_reg)
print("AUC-ROC (Régression Logistique) :\n", roc_auc_log_reg)


### 2 - Decision Tree
from sklearn.tree import DecisionTreeClassifier

# Initialisation et entraînement du modèle
tree_model = DecisionTreeClassifier(random_state=42)
tree_model.fit(X_train, y_train)

# Prédictions
y_pred_tree = tree_model.predict(X_test)

# Matrice de confusion
conf_matrix_tree = confusion_matrix(y_test, y_pred_tree)
print("Matrice de Confusion (Decision Tree) :\n", conf_matrix_tree)

# Rapport de classification
class_report_tree = classification_report(y_test, y_pred_tree)
print("Rapport de Classification (Decision Tree) :\n", class_report_tree)

# AUC-ROC
roc_auc_tree = roc_auc_score(y_test, y_pred_tree)
print("AUC-ROC (Decision Tree) :\n", roc_auc_tree)
### 3- Support Vector Machine (SVM)
from sklearn.svm import SVC


svm = SVC(probability=True)
svm.fit(X_train, y_train)


y_pred_svm = svm.predict(X_test)


# Matrice de confusion
conf_matrix_svm = confusion_matrix(y_test, y_pred_svm)
print("Matrice de Confusion :\n", conf_matrix_svm)

# Rapport de classification
class_report_svm = classification_report(y_test, y_pred_svm)
print("Rapport de Classification :\n", class_report_svm)

# AUC-ROC
roc_auc_svm = roc_auc_score(y_test, y_pred_svm)
print("AUC-ROC :\n", roc_auc_svm)














### 4 - Perceptron

In [ ]:
from sklearn.linear_model import Perceptron

# Initialisation et entraînement du modèle

perceptron = Perceptron(max_iter=1000, random_state=42, tol=1e-3)
perceptron.fit(X_train, y_train)

# Prédictions
y_pred_perceptron = perceptron.predict(X_test)

# Matrice de confusion
conf_matrix_perceptron = confusion_matrix(y_test, y_pred_perceptron)
print("Matrice de Confusion \n", conf_matrix_perceptron)

# Rapport de classification
class_report_perceptron = classification_report(y_test, y_pred_perceptron)
print("Rapport de Classification:\n", class_report_perceptron)

# AUC-ROC
roc_auc_perceptron = roc_auc_score(y_test, y_pred_perceptron)
print("AUC-ROC :\n", roc_auc_perceptron)

### 5 -Naive Bayès


In [ ]:
from sklearn.naive_bayes import GaussianNB

# Initialisation et entraînement du modèle
nb = GaussianNB()
nb.fit(X_train, y_train)

# Prédictions
y_pred_nb = nb.predict(X_test)

# Matrice de confusion
conf_matrix_nb = confusion_matrix(y_test, y_pred_nb)
print("Matrice de Confusion (Naive Bayes) :\n", conf_matrix_nb)

# Rapport de classification
class_report_nb = classification_report(y_test, y_pred_nb)
print("Rapport de Classification (Naive Bayes) :\n", class_report_nb)

# AUC-ROC
roc_auc_nb = roc_auc_score(y_test, y_pred_nb)
print("AUC-ROC (Naive Bayes) :\n", roc_auc_nb)


### 6. Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Initialisation et entraînement du modèle
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Prédictions
y_pred_rf = rf_model.predict(X_test)

# Matrice de confusion
conf_matrix_rf = confusion_matrix(y_test, y_pred_rf)
print("Matrice de Confusion (Random Forest) :\n", conf_matrix_rf)

# Rapport de classification
class_report_rf = classification_report(y_test, y_pred_rf)
print("Rapport de Classification (Random Forest) :\n", class_report_rf)

# AUC-ROC
roc_auc_rf = roc_auc_score(y_test, y_pred_rf)
print("AUC-ROC (Random Forest) :\n", roc_auc_rf)

### 7 - Modèle final
Notre modèle final est le svm et on l'exporte  avec `joblib`.

In [ ]:
joblib.dump(svm , "modele_svm.joblib")